In [ ]:
import matplotlib.pyplot as plt
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [ ]:
financials_2018 = pd.read_csv('../data/2018_Financial_Data.csv')

In [ ]:
financials_2018.Sector.unique()

In [ ]:
consumer_2018 = financials_2018.loc[financials_2018['Sector'] == "Consumer Cyclical"]

In [ ]:
consumer_2018

In [ ]:
X, y_true = consumer_2018(n_samples=100, centers=3,
                       cluster_std=0.60, random_state=0)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], s=50);

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)


In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='viridis')

centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);
#visualize results